In [1]:
from os.path import join
import numpy as np
import pandas as pd
from numpy import linalg as LA
from torch.autograd import Variable
import torchvision
import sys
import torch
import os
import scipy.io
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

## GPU or CPU
GPU = True
if GPU:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print(f"GPU(s): {torch.cuda.device_count()}x {torch.cuda.get_device_name()}")
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
    print("CPU")
eps = torch.finfo(torch.float32).eps
torch.cuda.empty_cache()

GPU(s): 1x NVIDIA GeForce RTX 3060


In [3]:
def np_to_tensor(Mat):
    return torch.from_numpy(Mat)

def np_to_var(Mat, dtype = torch.cuda.FloatTensor):
    return Variable(np_to_tensor(Mat)[None, :])

def Common_neighbours(V):
    return V @ V.T

# Read Dataset
def readDataset(selected):

    edgelist = np.genfromtxt(f'Dataset/Celegansneural.edges').astype(int)
    e = len(edgelist)
    n = np.max(edgelist)
    A = torch.zeros([n, n])
    Atr = torch.zeros([n, n])

    # Percent of delete edges
    per = 0.9
    oe = np.round(per * e).astype(int)

    train = np.random.choice(e, oe, replace=False)

    for i in range(e):
        A[edgelist[i, 0] - 1, edgelist[i, 1] - 1] = 1
        A[edgelist[i, 1] - 1, edgelist[i, 0] - 1] = 1

    for i in train:
        Atr[edgelist[i, 0] - 1, edgelist[i, 1] - 1] = 1
        Atr[edgelist[i, 1] - 1, edgelist[i, 0] - 1] = 1

    Ate = A - Atr
    dataset_name = 'Celegansneural'
    print("Dataset_name: ",dataset_name,"\n#node => ",n, "#edge => ",e,'\n')
    return A, Atr, Ate, dataset_name, n, e

# precision Mesure
def precision(V, Ate,Apred):
    
    n = Ate.shape[0]
    Ap = (1-V) * Apred
    L1 = torch.sum(Ate).int()
    
    ap = torch.reshape(Ap, (n**2,1))
    ate = torch.reshape(Ate, (n**2,1))
    values, indices = ap.topk(k=L1,dim=0) 
    Ind = indices[values!=0]
    app = torch.zeros(n**2,1)
    app[Ind] = 1
    
    l1 = torch.sum(app.cuda() * ate.cuda())
    prec = l1/L1
    return prec.cpu()

In [5]:
# Graph regularization weighted nonnegative matrix factorization for link prediction in weighted complex network (GWNMF)

A, Atr, Ate, dataset_name, n, e = readDataset(1)
[F,N] = A.shape
A = A.type(dtype)
Ate = Ate.type(dtype)
V = Atr.type(dtype)
# print ('Size of ATrain: ' ,V.shape)
degree = torch.sum(V , 0)
    
itere = 200
alpha = 10
betta = 1.9
K = 40

# Cosine Similarity

X1 = torch.norm(V , dim = 0, p = 2)
b = X1.repeat(n,1)
S = Common_neighbours(V) / (b.T * b)
S = torch.nan_to_num(S, nan=0)
D = torch.diag(torch.sum(S,0))

U = abs(np.random.normal(0,1, size = (N,K)))
U = np_to_var(U)[0].type(dtype)

V0 = abs(np.random.normal(0,1,size = (N,K)))
V0 = np_to_var(V0)[0].type(dtype)

epsU = torch.ones([N,K]) * torch.finfo(torch.float32).eps
epsV0 = torch.ones([N,K]) * torch.finfo(torch.float32).eps

for i in range(1,itere):

    Uup = (S * V) @ V0
    Udo = (S * (U @ V0.T)) @ V0 + 2* betta * U
    U   = U * (Uup / (Udo + torch.finfo(torch.float32).eps))

    V0up = (S * V).T @ U + alpha * (S @ V0)
    V0do = (S *(S *(U @ V0.T)).T) @ U + alpha * (D @ V0)+ betta * V0
    V0  = V0 * (V0up / (V0do + torch.finfo(torch.float32).eps))


Apred = U @ V0.T
prec = precision(V, Ate, Apred)
print('GWNMF Precision= ',prec)

Dataset_name:  Celegansneural 
#node =>  297 #edge =>  2345 

GWNMF Precision=  tensor(0.0550)
